In [2]:
pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=d25c6a31ad8adfcc4958b29750fdff56aef1bafcad52d74da5de0f04265c9b5a
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=0841d844a980ec8325a8abf97457f8e3e258d46196f77b81e8995eba031facf0
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1b

In [3]:
from newspaper import Article
import random
import string
import numpy as np
import warnings
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
warnings.filterwarnings('ignore')

In [5]:
nltk.download('punkt',quiet=True)
nltk.download('wordnet',quiet=True)

True

In [7]:
article= Article('https://simple.wikipedia.org/wiki/Astronomy')
article.download()
article.parse()
article.nlp()
corpus=article.text
#print
print(corpus)

Astronomy is the scientific study of celestial bodies. Stars, galaxies, planets, moons, asteroids, comets and nebulae are studied, as are supernovae explosions, gamma ray bursts, and cosmic microwave background radiation. Astronomy includes the development, physics, chemistry, meteorology and movement of celestial bodies. The big questions are the structure and development of the universe.

Astronomy is one of the oldest sciences. The patterns in the night sky were called constellations by the Arabs. They used the positions of the stars to navigate, and to find when was the best time to plant crops.

Astrophysics is an important part of astronomy. A related subject, cosmology, is concerned with studying the universe as a whole,[1] and the way the universe changed over time. Astronomy is not the same as astrology, a belief that the motion of the stars and the planets may affect human lives.

There are two main types of astronomy, observational and theoretical astronomy. Observational as

In [8]:
#tokenization
text=corpus
sent_tokens=nltk.sent_tokenize(text)
print(sent_tokens)

['Astronomy is the scientific study of celestial bodies.', 'Stars, galaxies, planets, moons, asteroids, comets and nebulae are studied, as are supernovae explosions, gamma ray bursts, and cosmic microwave background radiation.', 'Astronomy includes the development, physics, chemistry, meteorology and movement of celestial bodies.', 'The big questions are the structure and development of the universe.', 'Astronomy is one of the oldest sciences.', 'The patterns in the night sky were called constellations by the Arabs.', 'They used the positions of the stars to navigate, and to find when was the best time to plant crops.', 'Astrophysics is an important part of astronomy.', 'A related subject, cosmology, is concerned with studying the universe as a whole,[1] and the way the universe changed over time.', 'Astronomy is not the same as astrology, a belief that the motion of the stars and the planets may affect human lives.', 'There are two main types of astronomy, observational and theoretica

In [9]:
#removing the punctuation
remove_punct_dict=dict( (ord(punct),None) for punct in string.punctuation)
print(string.punctuation)
print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [10]:
#returning lower case words
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))
print(LemNormalize(text))

['astronomy', 'is', 'the', 'scientific', 'study', 'of', 'celestial', 'bodies', 'stars', 'galaxies', 'planets', 'moons', 'asteroids', 'comets', 'and', 'nebulae', 'are', 'studied', 'as', 'are', 'supernovae', 'explosions', 'gamma', 'ray', 'bursts', 'and', 'cosmic', 'microwave', 'background', 'radiation', 'astronomy', 'includes', 'the', 'development', 'physics', 'chemistry', 'meteorology', 'and', 'movement', 'of', 'celestial', 'bodies', 'the', 'big', 'questions', 'are', 'the', 'structure', 'and', 'development', 'of', 'the', 'universe', 'astronomy', 'is', 'one', 'of', 'the', 'oldest', 'sciences', 'the', 'patterns', 'in', 'the', 'night', 'sky', 'were', 'called', 'constellations', 'by', 'the', 'arabs', 'they', 'used', 'the', 'positions', 'of', 'the', 'stars', 'to', 'navigate', 'and', 'to', 'find', 'when', 'was', 'the', 'best', 'time', 'to', 'plant', 'crops', 'astrophysics', 'is', 'an', 'important', 'part', 'of', 'astronomy', 'a', 'related', 'subject', 'cosmology', 'is', 'concerned', 'with', '

In [11]:
#greetings
greeting_input=["hi","hello","hey","hola"]
greeting_response=["howdy","hey there","hi","hello :)"]
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in greeting_input:
      return random.choice(greeting_response)

In [12]:
def response(user_response):
 #user response and robo responce
  #user_response="WHat is chronic disease"
  user_response=user_response.lower()
  #print(user_response)
  #robo response
  robo_response=''
  sent_tokens.append(user_response)
  #print(sent_tokens)
  tfidfvec=TfidfVectorizer(tokenizer=LemNormalize , stop_words='english')
  tfidf=tfidfvec.fit_transform(sent_tokens)
  #print(tfidf)
  #get similarity score
  val=cosine_similarity(tfidf[-1],tfidf)
  #print(val)
  idx=val.argsort()[0][-2]
  flat=val.flatten()
  flat.sort()
  score=flat[-2]
  #print(score)
  if score==0:
    robo_response=robo_response+"sorry,i dont understand"
  else:
    robo_response=robo_response+sent_tokens[idx]

  sent_tokens.remove(user_response)
  return robo_response

In [17]:
greeting_input=["hi","hello","hey","hola"]
greeting_response=["howdy","hey there","hi","hello :)"]
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in greeting_input:
      return random.choice(greeting_response)
flag=True
print("Hello I am Starlight!!! I know stuff about astronomy, try asking :) (type bye to exit)")
while(flag==True):
  user_response=input("User:")
  #user_response=user_response.lower()
  if(user_response!='bye'):
    if(user_response=='thanks' or user_response=='thank you'):
      flag=False
      print("Starlight: anytime :)")
    else:
       if( greeting(user_response) != None):
         print("Starlight: "+ greeting(user_response))
       else:
         print("Starlight:"+response(user_response))
  else:
    flag=False
    print("Starlight: see you later :)")

Hello I am Starlight!!! I know stuff about astronomy, try asking :) (type bye to exit)
User:hello
Starlight: hi
User:what is atronomy?
Starlight:sorry,i dont understand
User:what is astronomy
Starlight:There are two main types of astronomy, observational and theoretical astronomy.
User:what is theoretical astronomy
Starlight:Theoretical astronomy explains what we see.
User:and observational
Starlight:There are two main types of astronomy, observational and theoretical astronomy.
User:what is observational astronomy
Starlight:There are two main types of astronomy, observational and theoretical astronomy.
User:what is solar astronomy
Starlight:Solar astronomy [ change | change source ]

See the main article: Sun

Solar astronomy is the study of the Sun.
User:bye
Starlight: see you later :)
